In [1]:
import re
import pandas as pd
pd.options.display.max_rows = None
import datetime
import time

In [2]:
# 開啟檔案
def open_brd_file(filepath):
    f = open(filepath)
    return f

# 找到ballname
def findBallName(ballName, string):
    result = re.findall(f"(UCPU1\.{ballName}\,|UCPU1\.{ballName}$)", string, re.I)
    return result

In [4]:
#step1: 開啟檔案
filepath = "Project TXT/ORB_ADL_LPDDR5_1125b.txt"
try:
    f = open_brd_file(filepath)
    print(f"step1: Open file. (File name: {filepath}) -DONE")
except:
    print(f"File is not exit, check file name.")

rawData = []
for i in f:
    row = i.replace("\n","")#去掉換行符號
    rawData.append(row)

step1: Open file. (File name: Project TXT/ORB_ADL_LPDDR5_1125b.txt) -DONE


In [5]:
#找出個$NETS並清除不需要的資料
keyIDX = []
for i in rawData:
    if re.findall(f"\$(NETS|END)", i , re.I):
#         print(re.findall(f"\$.*", i, re.I))
        keyIDX.append(rawData.index(i))
rawData2 = rawData[keyIDX[0] +1: keyIDX[1]]

In [6]:
# 把TXT整理成dictionary
start_time = time.time()
netName, netData = "", ""
netInfo = {}
clearData = {}
# 處理txt檔 分好net name 跟 GPIO資料
for line in rawData2:
#     print(re.findall(f".+\;", line))
    if re.findall(f".+\;", line):
        netName = re.findall(f".+\;", line)[0].replace(";", "")#分號前是net name
        
        #處理 net data 
        netData = re.findall(f"\;.+", line)[0].replace(";", "")#分號後是net data
        netData = netData.split()
        netData = ",".join(netData)


        netInfo = { netName : netData }
        clearData.update(netInfo)
    else:
        line = ",".join(line.split())#用空格白切割 再用,分割
        
        if clearData.__contains__(netName):
            clearData[netName] += line
        else:
            print("erroe", line)

end_time = time.time()
print ("執行時間2：", (end_time - start_time))

執行時間2： 0.12648510932922363


In [8]:
# 叫出intel GPIO template 
data = pd.read_excel('INTEL materials/GPIO2020.xlsx')
CPUBallName = data.iloc[:,0].tolist()

In [9]:
CPUBallName

['EV6',
 'EV8',
 'ET51',
 'R9',
 'U9',
 'W9',
 'DL8',
 'AH32',
 'AL3',
 'AM1',
 'EK26',
 'EK23',
 'FB56',
 'EL53',
 'EF56',
 'EF57',
 'EF59',
 'EC59',
 'EC61',
 'EG58',
 'EF61',
 'EG56',
 'FA53',
 'FC53',
 'EN53',
 'EK53',
 'AK35',
 'FA55',
 'EV58',
 'EV56',
 'EH53',
 'AK32',
 'L6',
 'L8',
 'DG3',
 'EH51',
 'DG1',
 'DV11',
 'DV10',
 'N8',
 'U6',
 'AA8',
 'W6',
 'FB6',
 'R8',
 'AH22',
 'AK17',
 'AJ15',
 'AH17',
 'AG15',
 'AD11',
 'AC12',
 'AA12',
 'AF37',
 'AH35',
 'AF35',
 'AH37',
 'AH25',
 'AF20',
 'AD16',
 'AA16',
 'F8',
 'AF22',
 'AF17',
 'DY1',
 'DY3',
 'EH26',
 'EF26',
 'EE26',
 'AF12',
 'AH12',
 'AK12',
 'AL12',
 'DY44',
 'EL51',
 'EN51',
 'DR61',
 'EN43',
 'EL43',
 'EH43',
 'EF43',
 'EH46',
 'EF46',
 'ET48',
 'ET43',
 'ET41',
 'ER41',
 'EN41',
 'ET46',
 'ER48',
 'ER46',
 'EL46',
 'EK46',
 'EL41',
 'EK41',
 'EN48',
 'EN46',
 'EL48',
 'EK48',
 'ET28',
 'EN28',
 'EH36',
 'EF36',
 'ET31',
 'ER31',
 'EL28',
 'EK31',
 'EH31',
 'EH28',
 'EN31',
 'EF31',
 'ET36',
 'ER33',
 'EL33',
 'EN3

In [471]:
data["netName"] = ""

In [472]:
for netName, netData in clearData.items():
#     print(netData)
    for ballName in CPUBallName:
        result = findBallName(ballName, netData)
#         print(result)
        if result:
            if data.loc[data[data["CPU Ball Name"]==f"{ballName}"].index.tolist()[0], "netName" ] == "":
#                 print("yes")
                data.loc[data[data["CPU Ball Name"]==f"{ballName}"].index.tolist()[0], "netName" ] = netName
            else:
            
                print("重複")
                data.loc[data[data["CPU Ball Name"]==f"{ballName}"].index.tolist()[0], "netName" ] += netName


In [473]:
elsxPath = f'GPIO mapping_20201228.xlsx'
write = pd.ExcelWriter(elsxPath)

# save_excel(write, dfFunction, "dfFunction")
data.to_excel(write, sheet_name=f'data', index=False)

write.save()